In [1]:
import sys
sys.path.insert(0, '/home/eddieh00/UCSD/ms/ece228/bmi_model/facenet/src')
import facenet
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split


In [2]:
def prewhiten(x):
    mean = np.mean(x)
    std = np.std(x)
    std_adj = np.maximum(std, 1.0 / np.sqrt(x.size))
    y = np.multiply(np.subtract(x, mean), 1 / std_adj)
    return y

def load_and_preprocess_image(image_path, image_size):
    try:
        # Load image
        image = Image.open(image_path)
        image = image.resize(image_size)
        image = np.array(image)

        # Preprocess the image
        image = prewhiten(image)

        return image
    except FileNotFoundError:
        print("File not found: " + str(image_path))
        return None

def generate_embeddings(image_paths, labels, model_path, image_size):
    assert len(image_paths) == len(labels), "Number of image paths and labels must be the same."
    
    with tf.Graph().as_default():
        with tf.Session() as sess:
            # Load the facenet model
            facenet.load_model(model_path)

            # Get input and output tensors
            input_tensor = tf.get_default_graph().get_tensor_by_name("input:0")
            output_tensor = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_tensor = tf.get_default_graph().get_tensor_by_name("phase_train:0")

            # Generate embeddings and matching labels
            embeddings = []
            matching_labels = []
            for image_path, label in zip(image_paths, labels):
                image = load_and_preprocess_image(image_path, image_size)
                if image is not None:
                    image = np.expand_dims(image, axis=0)
                    emb = sess.run(output_tensor, feed_dict={input_tensor: image, phase_train_tensor: False})
                    embeddings.append(emb.flatten())
                    matching_labels.append(label)

            embeddings = np.array(embeddings)

    return embeddings, matching_labels


def generate_test_embeddings(image_paths, model_path, image_size):
    with tf.Graph().as_default():
        with tf.Session() as sess:
            # Load the facenet model
            facenet.load_model(model_path)

            # Get input and output tensors
            input_tensor = tf.get_default_graph().get_tensor_by_name("input:0")
            output_tensor = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_tensor = tf.get_default_graph().get_tensor_by_name("phase_train:0")

            # Generate embeddings and matching labels
            embeddings = []
            for image_path in image_paths:
                image = load_and_preprocess_image(image_path, image_size)
                if image is not None:
                    image = np.expand_dims(image, axis=0)
                    emb = sess.run(output_tensor, feed_dict={input_tensor: image, phase_train_tensor: False})
                    embeddings.append(emb.flatten())

            embeddings = np.array(embeddings)

    return embeddings


In [5]:
from bmi_custom_model2 import BMI_Estimator

path = '/home/eddieh00/UCSD/ms/ece228/bmi_model/bmi_model/saved models/epoch_38'
with tf.Session() as sess:
    test = BMI_Estimator()
    checkpoint = tf.train.latest_checkpoint(path)
    if checkpoint:
        test.load(sess, checkpoint)
        
    # Prepare your image tensor
    test_image = '/home/eddieh00/UCSD/ms/ece228/bmi_model/data/test_images2_cropped/cropped.jpg'
    facenet_path = '/home/eddieh00/UCSD/ms/ece228/bmi_model/facenet/best_facenet_model'
    testimg = generate_test_embeddings([test_image], model_path=facenet_path, image_size=(160, 160))
    imgtensor = tf.convert_to_tensor(testimg, dtype=tf.float32)

    # Make a prediction
    output_tensor = test.call(imgtensor)

    # Evaluate the output tensor and get the actual output
    output_value = sess.run(output_tensor)
    print(output_value)

INFO:tensorflow:Restoring parameters from /home/eddieh00/UCSD/ms/ece228/bmi_model/bmi_model/saved models/epoch_38/model_weights.ckpt
Model directory: /home/eddieh00/UCSD/ms/ece228/bmi_model/facenet/best_facenet_model
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from /home/eddieh00/UCSD/ms/ece228/bmi_model/facenet/best_facenet_model/model-20180402-114759.ckpt-275
[[22.036179]]


In [1]:
import image_crop_processing
import matplotlib.pyplot as plt
from PIL import Image

image_dir = '/home/eddieh00/UCSD/ms/ece228/bmi_model/lol_dataset'
cropped_dir = '/home/eddieh00/UCSD/ms/ece228/bmi_model/data/lol_cropped'
image_crop_processing.crop_faces(image_dir, cropped_dir)



Cropping faces and saving to /home/eddieh00/UCSD/ms/ece228/bmi_model/data/lol_cropped


In [ ]:

plt.figure(figsize=(5,5))
plt.imshow()
plt.axis('off')  # to turn off axes for the image
plt.show()
